<a href="https://colab.research.google.com/github/ayanosk/rekihaku_engishiki_tei/blob/main/%E5%BB%B6%E5%96%9C%E5%BC%8FTEI_IIIF%E7%B4%90%E4%BB%98%E3%81%91%E7%94%A8%E3%82%B3%E3%83%BC%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from bs4 import BeautifulSoup
import json
import requests
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 関数

def createXml(manifest, id_format="{}"):

  baseText = r'''
        <facsimile></facsimile>
  '''

  soup = BeautifulSoup(baseText, "xml")

  facsimile = soup.find("facsimile")
  facsimile["source"] = manifest

  # manifestのロード

  manifest_data = requests.get(manifest).json()
  canvases = manifest_data["sequences"][0]["canvases"]

  for canvas in canvases:
      prefix = canvas["images"][0]["resource"]["service"]["@id"]
      image = prefix + "/full/full/0/default.jpg"
      canvasId = canvas["@id"]

      # canvasのIDの末尾をxml:idとして扱います。データや用途に応じて、適宜変更してください。
      id = id_format.format(canvasId.split("/")[-1])

      surface_text = '''
      <surface source="{}" xml:id="{}">
      <graphic url="{}"/>
      </surface>
      '''.format(canvasId, id, image)

      surface = BeautifulSoup(surface_text, "xml")
      facsimile.append(surface)

  return soup

def main(manifest, output_file, id_format="{}"):
  # 以下、実行
  xml = createXml(manifest, id_format)

  f = open(output_file, "w")
  f.write(xml.prettify())
  f.close()

  # ファイルのダウンロード
  from google.colab import files
  files.download(output_file)

In [6]:
# 延喜式（国立歴史民俗博物館）の例
manifest = "https://khirin-a.rekihaku.ac.jp/iiif/rekihaku/H-743-74-41/manifest.json"
output_file = "engishiki_41.xml"
main(manifest, output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
#graphic内のURLからfacs="{}"を生成する
xmlfile = open('/content/engishiki_41 (2).xml', "r", encoding='utf-8')
soup = BeautifulSoup(xmlfile, "xml")

graphics = soup.select("graphic")
urls = []
for graphic in graphics:
  facsurl = 'facs="{}"'.format(graphic["url"])
  urls.append(facsurl)

In [12]:
urls

['facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00001.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00002.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00003.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00004.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00005.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00006.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00007.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00008.tif/full/full/0/default.jpg"',
 'facs="https://khirin-a.rekihaku.ac.jp/iiif/2/engishiki%2FH-743-74-41%2F00009.tif/full/full/0/default.jpg"',
 'facs="ht

In [10]:
type(soup)

bs4.BeautifulSoup